# File I/O in Python

This notebook will cover the following topics:

1. Opening and closing files
2. Reading and writing text files
3. Reading and writing binary files
4. Using the `with` statement
5. Using the `pickle` module
6. Serializing objects with `pickle`
7. Reading and writing JSON files

## Opening and closing files

Let's start with the basics. To open a file, we use the built-in `open()` function. The `open()` function takes two arguments: the name of the file to open and the mode in which to open it. The mode can be one of the following:

* `'r'` - open for reading (default)
* `'w'` - open for writing, truncating the file first
* `'x'` - open for exclusive creation, failing if the file already exists
* `'a'` - open for writing, appending to the end of the file if it exists
* `'b'` - binary mode
* `'t'` - text mode (default)
* `'+'` - open a disk file for updating (reading and writing)

To make this tutorial more interesting, let's create a purpose for our code. Our goal is to create a logging system for players and their rolls. Every time a player rolls, we will add it to our log. We will also add a timestamp to each roll. We will store the log in a file called `log.txt`.

In [4]:
import random
import datetime

# First, we'll open a file
fp = open("log.txt", "w")

# Let's create a list of players
players = ["Naomi", "James", "Amos", "Bobbie"]

# Let's roll a d20 for each player and write it in the log, including a timestamp
for player in players:
    roll = random.randint(1, 20)
    timestamp = datetime.datetime.now()
    fp.write(f"{timestamp} - {player} rolled a {roll}\n")

# Finally, let's close the file
fp.close()

It isn't recommended to open and close the file manually like this. Instead, we will use the `with` statement. This will ensure that the file is closed properly even if an exception is raised.

Additionally, it is possible that calling `write` without using `with` will not write to the file immediately. Instead, it will be buffered and written later. Using `with` will ensure that the file is written to immediately.

```python
import datetime

with open('log.txt', 'a') as f:
    f.write(f'{datetime.datetime.now()}: {player} rolled {roll}\n')
```

## Reading and writing binary files

For the next example, we will write the user's rolls as binary. Each user will get their own file so that we can easily read their rolls later.

In [8]:
for player in players:
    rolls = [random.randint(1, 20) for _ in range(5)]
    with open("rolls/" + player + ".bin", "wb") as fp:
        fp.write(bytes(rolls))

# To verify, let's open the files and print the contents
for player in players:
    with open("rolls/" + player + ".bin", "rb") as fp:
        print(player, list(fp.read()))

Naomi [3, 16, 8, 7, 10]
James [1, 12, 18, 6, 3]
Amos [13, 8, 12, 14, 2]
Bobbie [7, 11, 19, 4, 12]
